In [1]:
import math
import time
import os
import json

import torch as T
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt

In [10]:
import motion_capture

from motion_capture.model import primaryModels


model = primaryModels.FullConvModel(128, 3, 10)

model(T.rand(5, 3, 128, 128))

running with neck: <class 'motion_capture.model.convolution.necks.UpsampleCrossAttentionrNeck'>


tensor([[[ 0.2244, -0.6158,  0.2961],
         [ 0.4649, -0.6930,  0.2423],
         [ 0.4447, -0.6111,  0.3488],
         [ 0.4098, -0.7241,  0.2867],
         [ 0.5571, -0.4692,  0.3012],
         [ 0.4822, -0.7920,  0.2826],
         [ 0.5228, -0.5817,  0.4432],
         [ 0.4856, -0.7587,  0.4615],
         [ 0.6528, -0.4922,  0.3782],
         [ 0.4061, -0.6727,  0.1855]],

        [[-0.4781, -0.7350, -0.2535],
         [-0.1808, -0.8107, -0.2508],
         [-0.1982, -0.6430, -0.1063],
         [-0.2696, -0.7685, -0.2514],
         [-0.1437, -0.5232, -0.2280],
         [-0.1951, -0.8496, -0.2316],
         [-0.1323, -0.6956, -0.0557],
         [-0.2099, -0.7994, -0.0378],
         [-0.0170, -0.6205, -0.1340],
         [-0.3126, -0.7231, -0.3310]],

        [[ 0.0818,  0.1004, -0.0076],
         [ 0.3618,  0.0725,  0.0425],
         [ 0.3894,  0.0723,  0.1568],
         [ 0.3320,  0.0793,  0.0485],
         [ 0.4595,  0.3672,  0.0703],
         [ 0.3839,  0.0294,  0.0574],
        

## Interesting Models

In [16]:
most_interesting_models = {
    "vit_b_32": models.get_model("vit_b_32").to("cuda"),
}

models.quantization
models.detection
models.segmentation

<module 'torchvision.models.detection' from 'c:\\Users\\noone\\Documents\\programming\\homemade-motion-capture\\.venv\\lib\\site-packages\\torchvision\\models\\detection\\__init__.py'>

## PyTorch Pretrained Models Speedtest

In [18]:
segmentation_models = {
    "fcn_resnet50": models.segmentation.fcn_resnet50(weights=models.segmentation.FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1),
    "fcn_resnet101": models.segmentation.fcn_resnet101(weights=models.segmentation.FCN_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1),
    "deeplabv3_resnet50": models.segmentation.deeplabv3_resnet50(weights=models.segmentation.DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1),
    "deeplabv3_resnet101": models.segmentation.deeplabv3_resnet101(weights=models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1),
    "deeplabv3_mobilenet_v3_large": models.segmentation.deeplabv3_mobilenet_v3_large(weights=models.segmentation.DeepLabV3_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1),
    "lraspp_mobilenet_v3_large": models.segmentation.lraspp_mobilenet_v3_large(weights=models.segmentation.LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1),
}

object_detection_models = {
    "fcos_resnet50_fpn": models.detection.fcos_resnet50_fpn(weights=models.detection.FCOS_ResNet50_FPN_Weights.COCO_V1),
    "fasterrcnn_mobilenet_v3_large_320_fpn": models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(weights=models.detection.FasterRCNN_MobileNet_V3_Large_320_FPN_Weights.COCO_V1),
    "fasterrcnn_mobilenet_v3_large_fpn": models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights=models.detection.FasterRCNN_MobileNet_V3_Large_FPN_Weights.COCO_V1),
    "fasterrcnn_resnet50_fpn_v2": models.detection.fasterrcnn_resnet50_fpn_v2(weights=models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.COCO_V1),
    "fasterrcnn_resnet50_fpn": models.detection.fasterrcnn_resnet50_fpn(weights=models.detection.FasterRCNN_ResNet50_FPN_Weights.COCO_V1),
    "retinanet_resnet50_fpn_v2": models.detection.retinanet_resnet50_fpn_v2(weights=models.detection.RetinaNet_ResNet50_FPN_V2_Weights.COCO_V1),
    "retinanet_resnet50_fpn": models.detection.retinanet_resnet50_fpn(weights=models.detection.RetinaNet_ResNet50_FPN_Weights.COCO_V1),
    "ssd300_vgg16": models.detection.ssd300_vgg16(weights=models.detection.SSD300_VGG16_Weights.COCO_V1),
    "ssdlite320_mobilenet_v3_large": models.detection.ssdlite320_mobilenet_v3_large(weights=models.detection.SSDLite320_MobileNet_V3_Large_Weights.COCO_V1),
}

instance_segmentation_models = {
    "maskrcnn_resnet50_fpn": models.detection.maskrcnn_resnet50_fpn(weights=models.detection.MaskRCNN_ResNet50_FPN_Weights.COCO_V1),
    "maskrcnn_resnet50_fpn_v2": models.detection.maskrcnn_resnet50_fpn_v2(weights=models.detection.MaskRCNN_ResNet50_FPN_V2_Weights.COCO_V1),
}

keypoint_detection_models = {
    "keypointrcnn_resnet50_fpn": models.detection.keypointrcnn_resnet50_fpn(weights=models.detection.KeypointRCNN_ResNet50_FPN_Weights.COCO_V1),
}

## Facer

In [19]:
import facer

In [35]:
facer_models = {
    # "retinaface/mobilenet": facer.face_detector("retinaface/mobilenet", device="cuda"),
    # "retinaface/resnet": facer.face_detector("retinaface/resnet50", device="cuda"),
    "farl/lapa/448": facer.face_parser("farl/lapa/448", device="cuda"),
    "farl/celebm/448": facer.face_parser("farl/celebm/448", device="cuda"),
    "farl/ibug300w/448": facer.face_aligner("farl/ibug300w/448", device="cuda"),
    "farl/aflw19/448": facer.face_aligner("farl/aflw19/448", device="cuda"),
    "farl/wflw/448": facer.face_aligner("farl/wflw/448", device="cuda"),
    "farl/celeba/224": facer.face_attr("farl/celeba/224", device="cuda")
}